## Обзор данных

In [1]:
# импортируем библиотеки 
import pandas as pd 

from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя  
             'pwd': 'Sdf4$2;d-d30pp', # пароль  
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',  
             'port': 6432, # порт подключения  
             'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],  
                                                         db_config['pwd'],  
                                                         db_config['host'],  
                                                         db_config['port'], 
                                                         db_config['db']) # сохраняем коннектор 

# Подключаемся к БД
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Ознакокмимся с таблицами:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
# Формируем sql-запрос
query = ''' SELECT *
            FROM books
        '''

In [4]:
# Выполняем запрос и сохраняем результат  выполнения в DataFrame
#Sqlalchemy автоматически установит названия колонок  такими же, как у таблицы в БД

books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [5]:
query = ''' SELECT *
            FROM authors
        '''

In [6]:
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;


In [7]:
query = ''' SELECT *
            FROM publishers
        '''

In [8]:
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [9]:
query = ''' SELECT *
            FROM ratings
        '''

In [10]:
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [11]:
query = ''' SELECT *
            FROM reviews
        '''

In [12]:
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг, вышедших после 01.01.2000 г.

In [13]:
query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''

In [14]:
count_book = pd.io.sql.read_sql(query, con = engine)
count_book

,count
0,819


### Вывод:

Таким образом, количество книг, вышедших после 01.01.2000 года - 819  книг. 

##  Количество обзоров и средняя оценка для каждой книги

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [15]:
query = ''' SELECT DISTINCT(b.title),
                   COUNT(DISTINCT(rev.review_id)) as count_review, 
                   AVG(r.rating) as avg_rating
            FROM books as b 
            FULL OUTER JOIN  authors as a on b.author_id = a.author_id
            FULL OUTER JOIN ratings as r on b.book_id = r.book_id
            FULL OUTER JOIN reviews as rev on b.book_id = rev.book_id
            GROUP BY b.book_id
            ORDER BY count_review desc, avg_rating desc
            
        '''

In [16]:
count_review =  pd.io.sql.read_sql(query, con = engine)
count_review

,title,count_review,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Essential Tales and Poems,0,4.000000
997,Leonardo's Notebooks,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


### Вывод:

Таким образом, по таблице видно, что есть книги, которые более популярны и имеют высокую оценку (в пордяке убывания) представлены в таблице

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц необходимо, что бы исключить из анализа брошюры

In [17]:
query = ''' SELECT p.publisher,
                   COUNT(b.book_id) as count_book
            FROM books as b join publishers as p on b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50  
            GROUP BY p.publisher_id
            ORDER BY count_book desc
          
        '''

In [18]:
count_publish =  pd.io.sql.read_sql(query, con = engine)
count_publish

,publisher,count_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Ballantine Books (NY),1
330,Plaza y Janés,1
331,HarperCollinsPublishers,1
332,Random House: Modern Library,1


### Вывод:

Таким образом, издательство, которое выпустило наибольшее число книг толще 50 страниц-Penguin Books.

## Автор с самой высокой средней оценкой книг (книги с 50 и более оценками)

In [19]:
query = ''' SELECT a.author,
                   AVG (r.rating),  
                   COUNT (r.rating)
            FROM books as b join authors as a on b.author_id = a.author_id
            JOIN ratings as r on b.book_id = r.book_id
            GROUP BY a.author_id, r.book_id
            HAVING COUNT (r.rating) >= 50
            ORDER BY  AVG (r.rating) desc
           
        
        '''

In [20]:
best_author =  pd.io.sql.read_sql(query, con = engine)
best_author

,author,avg,count
0,J.K. Rowling/Mary GrandPré,4.414634,82
1,J.R.R. Tolkien,4.391892,74
2,J.K. Rowling/Mary GrandPré,4.287500,80
3,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
4,J.K. Rowling/Mary GrandPré,4.246575,73
5,Louisa May Alcott,4.192308,52
6,J.K. Rowling/Mary GrandPré,4.186667,75
7,J.R.R. Tolkien,4.125000,88
8,Rick Riordan,4.080645,62
9,William Golding,3.901408,71


### Вывод:

Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками)-J.K. Rowling/Mary GrandPré

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [21]:
query = ''' with 
            i as 
            (SELECT username
            FROM ratings --as r  join reviews as rev on r.book_id = rev.book_id
            GROUP BY username
            HAVING COUNT(rating) > 50),
            
            m as 
            (SELECT username,
            COUNT(review_id) as count_review
            FROM reviews 
            GROUP BY username)
           
           
            SELECT AVG(m.count_review)
            FROM i LEFT OUTER JOIN m ON i.username=m.username

        '''
 

In [22]:
avg_review =  pd.io.sql.read_sql(query, con = engine)
avg_review

,avg
0,24.333333


### Вывод:

Таким образом, среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

## Выводы:

В ходе исследования, мы установили, что:
1. В нашем распоряжении больше всего книг, выпущенных после 01.01.2000 года (819 книг, общее количество книг- 1000 книг)
2. Для каждой книги посчитали количество обзоров и среднюю оценку.Более детальная характеристика по каждой книге представлена в таблице. 
3. Определили издательство, которое выпустило наибольшее число книг толще 50 страниц, что бы исключить из анализа издательства, которые выпускали брошюры и выяснили, что: издательство, которое выпустило наибольшее число книг толще 50 страниц-Penguin Books. 
4. Определили автора с самой высокой средней оценкой книг, у которых было 50 и более оценок-Diana Gabaldon..
5. Определили среднее количество обзоров от пользователей, которые поставили больше 50 оценок-24 обзора

    